# Provider Comparison with UnifiedLLM

One of the key benefits of UnifiedLLM is the ability to switch between different LLM providers using the same code interface.

**Topics covered:**
- Using the same code with different providers
- Comparing responses from Gemini, Anthropic, and OpenAI
- Handling missing API keys gracefully
- Understanding provider differences

**Supported providers:**
- 🔷 **Gemini** (Google) - Free tier available
- 🟣 **Claude** (Anthropic) - Pay-as-you-go
- 🟢 **GPT** (OpenAI) - Pay-as-you-go

## Setup

### API Keys
You'll need API keys for the providers you want to test:

- **Gemini**: `GEMINI_API_KEY` - Get at [Google AI Studio](https://aistudio.google.com/app/apikey)
- **Anthropic**: `ANTHROPIC_API_KEY` - Get at [Anthropic Console](https://console.anthropic.com/)
- **OpenAI**: `OPENAI_API_KEY` - Get at [OpenAI Platform](https://platform.openai.com/api-keys)

**Option 1 - Set via terminal (recommended):**
```bash
export GEMINI_API_KEY="your-google-key"
export ANTHROPIC_API_KEY="your-anthropic-key"
export OPENAI_API_KEY="your-openai-key"
```

**Option 2 - Set directly in notebook (if export doesn't work):**
```python
import os
os.environ["GEMINI_API_KEY"] = "your-google-key"
os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key"
os.environ["OPENAI_API_KEY"] = "your-openai-key"
```

**Note:** This notebook will skip providers whose API keys are not set.

In [ ]:
from unifiedllm import LLM
from unifiedllm.errors import MissingAPIKeyError, ProviderAPIError
import os

In [ ]:
# Option: Set API keys directly in notebook (if not already set via terminal)
# Uncomment and add your keys if the export command didn't work:
# os.environ["GEMINI_API_KEY"] = "your-google-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key"
# os.environ["OPENAI_API_KEY"] = "your-openai-key"

# Check which API keys are available
print("API Key Status:")
print("="*50)

keys_status = {
    "GEMINI_API_KEY": os.getenv("GEMINI_API_KEY"),
    "ANTHROPIC_API_KEY": os.getenv("ANTHROPIC_API_KEY"),
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")
}

for key_name, key_value in keys_status.items():
    status = "✅ Set" if key_value else "❌ Not set"
    print(f"{key_name}: {status}")

print("\nProviders with missing keys will be skipped in comparisons.")

## Define a Shared Prompt

Let's use the same prompt across all providers to compare their responses.

In [ ]:
# Shared prompt for comparison
test_prompt = "Explain the concept of machine learning in exactly 2 sentences."

# Alternative: use messages format
test_messages = [
    {"role": "user", "content": test_prompt}
]

## Helper Function for Provider Testing

This function will handle API calls and gracefully skip providers with missing keys.

In [ ]:
def run_provider(provider_name, model, env_var_name, prompt=None, messages=None):
    """
    Test a specific provider and return results.
    
    Args:
        provider_name: Name of the provider (e.g., "gemini", "anthropic", "openai")
        model: Model name for the provider
        env_var_name: Environment variable name for the API key
        prompt: Optional prompt string
        messages: Optional messages list
    
    Returns:
        Dictionary with results or error information
    """
    # Check if API key is set
    api_key = os.getenv(env_var_name)
    
    if not api_key:
        return {
            "provider": provider_name,
            "model": model,
            "status": "skipped",
            "reason": f"{env_var_name} not set",
            "text": None,
            "request_id": None,
            "usage": None
        }
    
    # Try to call the provider
    try:
        llm = LLM(provider=provider_name, model=model, api_key=None)
        
        if prompt:
            response = llm.chat(prompt=prompt)
        elif messages:
            response = llm.chat(messages=messages)
        else:
            raise ValueError("Must provide either prompt or messages")
        
        return {
            "provider": provider_name,
            "model": model,
            "status": "success",
            "text": response.text,
            "request_id": response.request_id,
            "usage": response.usage
        }
    
    except MissingAPIKeyError as e:
        return {
            "provider": provider_name,
            "model": model,
            "status": "error",
            "reason": f"Missing API key: {e}",
            "text": None,
            "request_id": None,
            "usage": None
        }
    
    except ProviderAPIError as e:
        return {
            "provider": provider_name,
            "model": model,
            "status": "error",
            "reason": f"API error: {e}",
            "text": None,
            "request_id": None,
            "usage": None
        }
    
    except Exception as e:
        return {
            "provider": provider_name,
            "model": model,
            "status": "error",
            "reason": f"Unexpected error: {e}",
            "text": None,
            "request_id": None,
            "usage": None
        }

## Run Comparison Across Providers

Let's test the same prompt with all three providers.

In [ ]:
# Define provider configurations
providers_config = [
    {
        "provider": "gemini",
        "model": "gemini-1.5-flash",
        "env_var": "GOOGLE_API_KEY",
        "display_name": "🔷 Gemini (Google)"
    },
    {
        "provider": "anthropic",
        "model": "claude-3-5-sonnet-20241022",
        "env_var": "ANTHROPIC_API_KEY",
        "display_name": "🟣 Claude (Anthropic)"
    },
    {
        "provider": "openai",
        "model": "gpt-4o-mini",
        "env_var": "OPENAI_API_KEY",
        "display_name": "🟢 GPT (OpenAI)"
    }
]

# Run comparison
results = []

print("Running provider comparison...\n")

for config in providers_config:
    print(f"Testing {config['display_name']}...")
    result = run_provider(
        provider_name=config["provider"],
        model=config["model"],
        env_var_name=config["env_var"],
        prompt=test_prompt
    )
    result["display_name"] = config["display_name"]
    results.append(result)

print("\n✅ Comparison complete!")

## Display Comparison Results

In [ ]:
def display_comparison(results):
    """
    Display comparison results in a readable format.
    """
    print("\n" + "="*80)
    print("PROVIDER COMPARISON RESULTS")
    print("="*80)
    
    for result in results:
        print(f"\n{result['display_name']}")
        print("-" * 80)
        print(f"Model: {result['model']}")
        print(f"Status: {result['status']}")
        
        if result['status'] == 'success':
            print(f"\nResponse:")
            print(result['text'])
            print(f"\nRequest ID: {result['request_id'] if result['request_id'] else 'N/A'}")
            
            if result['usage']:
                print(f"Token Usage: {result['usage']}")
            else:
                print("Token Usage: Not available")
        
        elif result['status'] == 'skipped':
            print(f"⚠️  Skipped: {result['reason']}")
        
        elif result['status'] == 'error':
            print(f"❌ Error: {result['reason']}")
    
    print("\n" + "="*80)

# Display the results
display_comparison(results)

## Comparison Summary Table

In [ ]:
def summary_table(results):
    """
    Create a simple text-based summary table.
    """
    print("\n" + "="*80)
    print("QUICK SUMMARY")
    print("="*80)
    print(f"{'Provider':<25} {'Status':<15} {'Response Length':<20}")
    print("-" * 80)
    
    for result in results:
        status_icon = {
            "success": "✅",
            "skipped": "⚠️",
            "error": "❌"
        }.get(result['status'], "❓")
        
        response_length = len(result['text']) if result['text'] else 0
        
        print(f"{result['display_name']:<25} {status_icon} {result['status']:<12} {response_length} chars")
    
    print("="*80)

summary_table(results)

## Testing with Message Format

Let's verify that the message-based approach works the same way across providers.

In [ ]:
# Define a multi-turn conversation
conversation_messages = [
    {"role": "user", "content": "What is 2 + 2?"}
]

print("Testing message-based chat across providers...\n")

for config in providers_config:
    result = run_provider(
        provider_name=config["provider"],
        model=config["model"],
        env_var_name=config["env_var"],
        messages=conversation_messages
    )
    
    print(f"{config['display_name']}:")
    if result['status'] == 'success':
        print(f"  Response: {result['text'][:100]}...")
    else:
        print(f"  {result['status'].upper()}: {result.get('reason', 'Unknown')}")
    print()

## Switching Providers in Your Code

Here's a practical example of how you might switch providers based on availability or preference.

In [ ]:
def get_available_llm():
    """
    Get the first available LLM provider.
    Priority: Gemini -> Anthropic -> OpenAI
    """
    providers = [
        ("gemini", "gemini-1.5-flash", "GEMINI_API_KEY"),
        ("anthropic", "claude-3-5-sonnet-20241022", "ANTHROPIC_API_KEY"),
        ("openai", "gpt-4o-mini", "OPENAI_API_KEY")
    ]
    
    for provider, model, env_var in providers:
        if os.getenv(env_var):
            print(f"✅ Using {provider} ({model})")
            return LLM(provider=provider, model=model, api_key=None)
    
    raise RuntimeError("No API keys found! Please set at least one: GEMINI_API_KEY, ANTHROPIC_API_KEY, or OPENAI_API_KEY")

# Example usage
try:
    my_llm = get_available_llm()
    response = my_llm.chat(prompt="Say hello!")
    print(f"\nResponse: {response.text}")
except RuntimeError as e:
    print(f"❌ {e}")

## Understanding Provider Differences

While UnifiedLLM provides a consistent interface, it's important to understand that different providers have unique characteristics:

### 🔷 Gemini (Google)
- **Pros**: Free tier available, fast responses, good for prototyping
- **Cons**: Smaller context window than some competitors
- **Best for**: Learning, prototyping, cost-conscious projects

### 🟣 Claude (Anthropic)
- **Pros**: Excellent reasoning, large context window, strong safety features
- **Cons**: Pay-as-you-go only, can be verbose
- **Best for**: Complex reasoning tasks, document analysis, safety-critical applications

### 🟢 GPT (OpenAI)
- **Pros**: Widely adopted, strong ecosystem, consistent performance
- **Cons**: Pay-as-you-go only, rate limits on free tier
- **Best for**: Production applications, well-established use cases

### Key Considerations

1. **Output Differences**: Different models will produce different responses to the same prompt, even if the quality is similar.

2. **Pricing**: Each provider has different pricing models. Gemini offers a free tier, while Anthropic and OpenAI are pay-as-you-go.

3. **Rate Limits**: Free tiers and paid tiers have different rate limits and quotas.

4. **Response Format**: While UnifiedLLM normalizes the response format, the underlying raw responses differ.

5. **Features**: Some providers support features others don't (e.g., function calling, vision capabilities).

## Practical Use Case: Fallback Pattern

A common pattern is to use a fallback provider if your primary one fails or is rate-limited.

In [ ]:
def chat_with_fallback(prompt, providers_list):
    """
    Try providers in order until one succeeds.
    
    Args:
        prompt: The prompt to send
        providers_list: List of (provider, model, env_var) tuples
    
    Returns:
        ChatResponse or None
    """
    for provider, model, env_var in providers_list:
        if not os.getenv(env_var):
            print(f"⚠️  Skipping {provider}: {env_var} not set")
            continue
        
        try:
            print(f"Trying {provider}...")
            llm = LLM(provider=provider, model=model, api_key=None)
            response = llm.chat(prompt=prompt)
            print(f"✅ Success with {provider}!")
            return response
        except Exception as e:
            print(f"❌ {provider} failed: {e}")
            continue
    
    print("❌ All providers failed!")
    return None

# Example usage
fallback_providers = [
    ("gemini", "gemini-1.5-flash", "GEMINI_API_KEY"),
    ("anthropic", "claude-3-5-sonnet-20241022", "ANTHROPIC_API_KEY"),
    ("openai", "gpt-4o-mini", "OPENAI_API_KEY")
]

response = chat_with_fallback("What is 5 + 7?", fallback_providers)
if response:
    print(f"\nFinal response: {response.text}")

## Summary

You've learned how to:
- ✅ Use the same code interface across multiple LLM providers
- ✅ Compare responses from different providers
- ✅ Handle missing API keys gracefully
- ✅ Implement fallback patterns for resilience
- ✅ Understand key differences between providers

**Key takeaway:** UnifiedLLM makes it easy to switch providers without changing your code structure, giving you flexibility in choosing the best provider for your needs, budget, and use case.

**Next steps:**
- Experiment with your own prompts across providers
- Build a simple app that lets users choose their preferred provider
- Implement cost optimization by routing simple queries to cheaper models